In [5]:
from utils import UNet, SegmentationDataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

dataset = SegmentationDataset("SA1B_Meta_AI_Segmentation_Dataset/")
dataloader = DataLoader(dataset, batch_size=50, shuffle=True)
model = UNet(in_channels=3, out_channels=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [6]:
def train_unet(model, dataset, epochs=10, lr=1e-4):
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()  # For binary segmentation

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for imgs, masks in dataloader:
            imgs, masks = imgs.cuda(), masks.cuda()  # Move to GPU if available

            # Forward pass
            preds = model(imgs)
            loss = criterion(preds, masks.unsqueeze(1))  # Add channel dim to masks
            epoch_loss += loss.item()

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")

    return model

In [7]:
model.train()
for epoch in range(2):
	epoch_loss = 0
	for idx, (image, mask) in enumerate(dataloader):
		outputs = model(image)
		loss = criterion(outputs, mask)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		epoch_loss += loss.item()
		# outputs.weights
		print(f'epoch: {epoch+1} | loss: {epoch_loss}')


Bounding Box: [711.0, 954.0, 154.0, 123.0]
Segmentation RLE: \_bP15f^13M3N2M4L3N3L4L4M6I9G<E<C9H3L4M1N2O000O1O1...
Predicted IoU: 0.8857464790344238
Bounding Box: [1823.0, 1134.0, 183.0, 288.0]
Segmentation RLE: k`_c2?\^13M3M2O2M2O1O2N1O2N1abNoNQ]1S1jbNSOS]1Q1gb...
Predicted IoU: 0.9431462287902832
Bounding Box: [1822.0, 1134.0, 93.0, 91.0]
Segmentation RLE: oQ^c2>]^14L2N3N1N2O2N1N2O2N1O2^bNROR]1P1jbNTOT]1[1...
Predicted IoU: 0.9308024644851685
Bounding Box: [1486.0, 993.0, 124.0, 120.0]
Segmentation RLE: fgQT28`^19I6J6K4L4L4L4L6K5K5K6J5L2M3M2N2O1N2N101N2...
Predicted IoU: 0.9552524089813232
Bounding Box: [1529.0, 865.0, 224.0, 129.0]
Segmentation RLE: oePV26d^15K3N2M3I6N3M2O1O100O101N101O1N2O1O1N2O000...
Predicted IoU: 0.9898009300231934
Bounding Box: [1280.0, 728.0, 290.0, 194.0]
Segmentation RLE: Qjcj1;\^19J3N3K3N3L3N3N1O2M201N1O2N101N1O1O2M2M3L4...
Predicted IoU: 0.9989508986473083
Bounding Box: [1476.0, 491.0, 129.0, 99.0]
Segmentation RLE: RcbS24f^16I6L3M4M4N101M2N3M2N3M2M2O2N1O1

TypeError: object of type 'builtin_function_or_method' has no len()

In [ ]:
print(dataset)